<a href="https://colab.research.google.com/github/nt189/Regresi-n-log-stica/blob/main/Regresi%C3%B3n_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [262]:
# Selecciona el pais
# pais = 'mexico'
# pais = 'california'
pais = 'ottawa'

# **Carga de librerias y csv**

In [263]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [264]:
df = pd.read_csv('listings_' + pais + '.csv')
df.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,261065,https://www.airbnb.com/rooms/261065,20250311200222,2025-03-12,city scrape,Westboro Village Executive Suite,A tastefully decorated and well equipped upper...,"Westboro Village is a century-old community, a...",https://a0.muscache.com/pictures/airflow/Hosti...,1369632,...,4.94,4.96,4.91,STR-825-691,f,1,1,0,0,0.67
1,290033,https://www.airbnb.com/rooms/290033,20250311200222,2025-03-12,city scrape,Rural charm close to the city,Close to Strathmere and other wedding venues. ...,Most folks ask about Strathmere - we are less ...,https://a0.muscache.com/pictures/3016522/1e453...,415201,...,4.92,5.00,4.92,STR-834-339,f,1,1,0,0,0.08
2,490182,https://www.airbnb.com/rooms/490182,20250311200222,2025-03-12,city scrape,Basement Studio - Old Ottawa East,NaN,"The worlds longest skating rink, a UNESCO heri...",https://a0.muscache.com/pictures/5a7e6363-5b91...,2401179,...,5.00,4.89,4.78,NaN,f,1,0,1,0,0.11


In [265]:
#Rellenamos nulos
df = df.fillna(method='bfill')
df = df.fillna(method='ffill')

<ipython-input-265-00d3f016ac00>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')
<ipython-input-265-00d3f016ac00>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [266]:
df.isnull().sum().to_frame(name='Cantidad de Nulos')

,Cantidad de Nulos
id,0
listing_url,0
scrape_id,0
last_scraped,0
source,0
...,...
calculated_host_listings_count,0
calculated_host_listings_count_entire_homes,0
calculated_host_listings_count_private_rooms,0
calculated_host_listings_count_shared_rooms,0


In [267]:
# Lista de columnas a limpiar
columnas_a_limpiar = ["price", "host_response_rate", "host_acceptance_rate"]

# Función para limpiar los valores
def limpiar_valor(valor):
    if isinstance(valor, str):  # Verificar si es un string
        return float(valor.replace("$", "").replace("%", "").replace(",", ""))
    return valor

# Aplicar limpieza a cada columna
for col in columnas_a_limpiar:
    df[col] = df[col].apply(limpiar_valor)

# **Regrecion logistica**

In [268]:
# Vars_indep = X    |    Var_dep = y (variable dicotomica)
def regresion_logistica(X, y, tbl_info=False):
  #Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

  # Se escalan todos los datos
  escalar = StandardScaler()

  # Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
  X_train = escalar.fit_transform(X_train)
  X_test = escalar.transform(X_test)

  algoritmo = LogisticRegression()

  # entrenamos el modelo
  algoritmo.fit(X_train, y_train)

  # Creo un diccionario
  resultados = {}

  # Realizamos una prediccion
  y_pred = algoritmo.predict(X_test)
  # y_pred

  # Verifico la matriz de Confusion
  matriz = confusion_matrix(y_test, y_pred)
  # print('Matriz de Confusion:')
  # print(matriz)

  # Calculo la precicion del modelo
  precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
  # print('Precision del modelo:')
  # print(precision)

  # Calculo de la exactitud del modelo
  exactitud = accuracy_score(y_test, y_pred)
  # print('Exactitud del modelo:')
  # print(exactitud)

  # Calculamos la sensibilidad del modelo
  sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=1)
  # print('Sensibilidad del modelo:')
  # print(sensibilidad)

  if tbl_info:
    tbl_info = pd.DataFrame({
            'Variable Dependiente': [y.name],
            'Variables Independientes': [", ".join(X.columns)],
            'Precisión': [precision],
            'Exactitud': [exactitud],
            'Sensibilidad': [sensibilidad],
            'Matriz de Confusión': [matriz.tolist()]
        })
    return tbl_info
  else:
    resultados['y_pred'] = y_pred
    resultados['matriz'] = matriz
    resultados['precision'] = precision
    resultados['exactitud'] = exactitud
    resultados['sensibilidad'] = sensibilidad

    return resultados



# **Convertir a dicotomicas**

In [269]:
import numpy as np

x = [
    'host_is_superhost',
    'host_identity_verified',
    'has_availability',
    'instant_bookable'
]

# Convertir 't' → 1 y 'f' → 0 usando map
for i in x:
    df[i] = df[i].map({'t': 1, 'f': 0})  # Convertir a 0 y 1

    unique_vals, counts = np.unique(df[i], return_counts=True)

    if len(unique_vals) == 1:  # Si solo tiene un único valor (todo 0s o todo 1s)
        total_rows = len(df)
        min_count = int(0.03 * total_rows)  # 3% del total

        if unique_vals[0] == 1:  # Si solo hay 1s, agregar ceros
            zero_indices = np.random.choice(df.index, size=min_count, replace=False)
            df.loc[zero_indices, i] = 0
            print(f"🔄 Ajustado '{i}': Se añadieron {min_count} ceros.")
        else:  # Si solo hay 0s, agregar unos
            one_indices = np.random.choice(df.index, size=min_count, replace=False)
            df.loc[one_indices, i] = 1
            print(f"🔄 Ajustado '{i}': Se añadieron {min_count} unos.")

    else:
        zero_count = (df[i] == 0).sum()
        one_count = (df[i] == 1).sum()
        total_rows = len(df)

        # Asegurar que hay al menos 3% de cada clase
        min_count = int(0.03 * total_rows)  # 3% del total

        if zero_count < min_count:
            zero_indices = np.random.choice(df[df[i] == 1].index, size=min_count - zero_count, replace=False)
            df.loc[zero_indices, i] = 0
            print(f"🔄 Ajustado '{i}': Se añadieron {min_count - zero_count} ceros.")

        if one_count < min_count:
            one_indices = np.random.choice(df[df[i] == 0].index, size=min_count - one_count, replace=False)
            df.loc[one_indices, i] = 1
            print(f"🔄 Ajustado '{i}': Se añadieron {min_count - one_count} unos.")

    print(f"{i}: {np.unique(df[i], return_counts=True)}")


host_is_superhost: (array([0, 1]), array([1601, 1235]))
host_identity_verified: (array([0, 1]), array([ 252, 2584]))
🔄 Ajustado 'has_availability': Se añadieron 85 ceros.
has_availability: (array([0, 1]), array([  85, 2751]))
instant_bookable: (array([0, 1]), array([2257,  579]))


In [270]:
print(np.unique(df['bathrooms']))
print(np.unique(df['bedrooms']))
print(np.unique(df['host_response_time']))
print(np.unique(df['room_type']))
print(np.unique(df['accommodates']))

[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  6.  7.5]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8. 10.]
['a few days or more' 'within a day' 'within a few hours' 'within an hour']
['Entire home/apt' 'Hotel room' 'Private room' 'Shared room']
[ 1  2  3  4  5  6  7  8  9 10 11 12 14 15 16]


In [271]:
# Tiene 0 baños
df['bathrooms'] = np.where(df['bathrooms'] == 0, '0 baños', 'No 0 baños')

df['bathrooms'] = df['bathrooms'].replace(['0 baños'], 1)
df['bathrooms'] = df['bathrooms'].replace(['No 0 baños'], 0)

print(df['bathrooms'].unique())

[0 1]


<ipython-input-271-260f65eb91f3>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['bathrooms'] = df['bathrooms'].replace(['No 0 baños'], 0)


In [272]:
# Tiene mas de 3 camas
df['bedrooms'] = np.where(df['bedrooms'] > 3, 'Mas de 3 camas', '3 camas o menos')

df['bedrooms'] = df['bedrooms'].replace(['Mas de 3 camas'], 1)
df['bedrooms'] = df['bedrooms'].replace(['3 camas o menos'], 0)

print(df['bedrooms'].unique())

[0 1]


<ipython-input-272-ccdcba50c09a>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['bedrooms'] = df['bedrooms'].replace(['3 camas o menos'], 0)


In [273]:
# Responde en menos de una hora
df['host_response_time'] = np.where(df['host_response_time'] == 'within an hour', 1, 0)

print(df['host_response_time'].unique())

[0 1]


In [274]:
# El tipo de habitacion es privada
df['room_type'] = np.where(df['room_type'] == 'Private room', 1, 0)

print(df['room_type'].unique())

[0 1]


In [275]:
# Tiene capacidad para 4 personas
df['accommodates'] = np.where(df['accommodates'] == 4, 1, 0)

print(df['accommodates'].unique())

[1 0]


In [276]:
media = df['price'].mean()

df['price'] = np.where(df['price'] > media, 'es mayor a la media', 'menor a la media')

df['price'] = df['price'].replace(['es mayor a la media'], 1)
df['price'] = df['price'].replace(['menor a la media'], 0)
print(df['price'].unique())

[1 0]


<ipython-input-276-167b379f2167>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['price'] = df['price'].replace(['menor a la media'], 0)


# **Variables**

In [277]:
# Variables dependientes
Variables_dependientes = [
    'host_is_superhost',
    'host_identity_verified',
    'has_availability',
    'instant_bookable',
    'bathrooms',
    'bedrooms',
    'host_response_time',
    'room_type',
    'accommodates',
    'price'
]

# Variables independientes (en el mismo orden que las dependientes)
Variables_independientes = [
    ['host_acceptance_rate', 'host_listings_count', 'review_scores_rating'],  # host_is_superhost
    ['host_has_profile_pic', 'host_listings_count', 'host_verifications'],  # host_identity_verified
    ['availability_30', 'availability_90', 'availability_365'],  # has_availability
    ['number_of_reviews', 'review_scores_accuracy', 'review_scores_cleanliness'],  # instant_bookable
    ['bathrooms_text', 'property_type', 'review_scores_cleanliness'],  # bathrooms
    ['beds', 'review_scores_value', 'availability_60'],  # bedrooms
    ['host_response_rate', 'number_of_reviews_l30d', 'review_scores_communication'],  # host_response_time
    ['review_scores_location', 'neighbourhood_group_cleansed'],  # room_type
    ['minimum_nights', 'maximum_nights', 'calculated_host_listings_count'],  # accommodates
    ['minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'review_scores_checkin']  # price
]


# **Convertir a numericas**

In [278]:
def numerica(x):
    mapping = dict(enumerate(df[x].astype("category").cat.categories))  # Guarda el mapeo de códigos
    print(f"Mapping para '{x}': {mapping}")
    df[x] = df[x].astype("category").cat.codes

for lista in Variables_independientes:  # Itera sobre cada lista dentro de la lista principal
    for columna in lista:  # Itera sobre cada variable dentro de la lista
        numerica(columna)


Mapping para 'host_acceptance_rate': {0: 0.0, 1: 2.0, 2: 7.0, 3: 9.0, 4: 10.0, 5: 12.0, 6: 14.0, 7: 17.0, 8: 18.0, 9: 22.0, 10: 25.0, 11: 26.0, 12: 28.0, 13: 29.0, 14: 33.0, 15: 34.0, 16: 36.0, 17: 38.0, 18: 40.0, 19: 42.0, 20: 43.0, 21: 48.0, 22: 49.0, 23: 50.0, 24: 54.0, 25: 55.0, 26: 56.0, 27: 57.0, 28: 58.0, 29: 59.0, 30: 60.0, 31: 61.0, 32: 62.0, 33: 63.0, 34: 64.0, 35: 65.0, 36: 66.0, 37: 67.0, 38: 68.0, 39: 69.0, 40: 70.0, 41: 71.0, 42: 72.0, 43: 73.0, 44: 74.0, 45: 75.0, 46: 76.0, 47: 77.0, 48: 78.0, 49: 79.0, 50: 80.0, 51: 81.0, 52: 82.0, 53: 83.0, 54: 84.0, 55: 85.0, 56: 86.0, 57: 87.0, 58: 88.0, 59: 89.0, 60: 90.0, 61: 91.0, 62: 92.0, 63: 93.0, 64: 94.0, 65: 95.0, 66: 96.0, 67: 97.0, 68: 98.0, 69: 99.0, 70: 100.0}
Mapping para 'host_listings_count': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 22, 19: 23, 20: 24, 21: 25, 22: 26, 23: 30, 24: 31, 25: 32, 26: 35, 27: 43, 28: 57, 29: 61, 30: 67,

# **Resultados**

In [279]:
from re import X
coeficientesTbl = pd.DataFrame()

for i in range(10):  # Iterar del 0 al 9
  y = Variables_dependientes[i]  # Variable dependiente
  X_cols = Variables_independientes[i]  # Variables independientes
  print(y)
  results = regresion_logistica(df[X_cols], df[y], tbl_info=True)  # Llamar a la función

  # Concatenar los resultados al DataFrame
  coeficientesTbl = pd.concat([coeficientesTbl, results], ignore_index=True)

# Guardar en CSV
coeficientesTbl.to_csv('coeficientes_' + pais + '.csv', index=False)

host_is_superhost
host_identity_verified
has_availability
instant_bookable
bathrooms


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


bedrooms
host_response_time
room_type
accommodates
price


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
